<a href="https://colab.research.google.com/github/BenjiElysium/SimpletonAgents-VerseGenerator/blob/main/SimpletonAgentsVerseGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Simpleton Agents Verse Generator**

---



This notebook demonstrates how to install the OpenAI library and obtain an API key for authentication. Once openai is installed the **Simpleton Agents Verse Generator** can be launched. 


---


It is recommend to keep the N setting to 1 as this performs the three stage verse generation and refinement process.

Created by Philip Benn: philipaldenbenn@gmail.com


In [ ]:
!pip install openai

In [ ]:
from IPython.display import clear_output
import getpass

print("Enter your OpenAI API key:")
api_key = getpass.getpass()
clear_output()
print("API key received.")

In [ ]:
import openai
from IPython.display import display, HTML
import textwrap

openai.api_key = api_key

def wrapped_print(text, width=80):
    wrapped_lines = textwrap.wrap(text, width=width)
    wrapped_text = "<br>".join(wrapped_lines)
    line_style = f"style='border-top: 1px solid; width: {width}ch; margin: 14px 0;'"
    display(HTML(f"<pre>{wrapped_text}</pre><div {line_style}></div><br>"))

def send_message(prompt, temperature, role, max_tokens):
    system_message = ""
    if role == "Poet":
        system_message = "You are a poet. Please immediately write a short poem in the style of Italo Calvino. There is no need to introduce yourself. "
    elif role == "Proof Reader":
        system_message = "You are a talented proof reader. Please immediately proof the poem provided by the 'Poet' in order to validate that all words are actual English language words. With any non-sensical words that you encounter, replace with an English word with a similar sound. There is no need to introduce yourself."
    elif role == "Editor":
        system_message = "You are an reknown editor. Please immediately create a new version of the poem that the 'Proof Reader' has offered you. Verify all words are in the English language. Your version should be organized and edited for solid structure and clear form. Do not repeat phrases or words. Be true to the original poem in this endeavor! There is no need to introduce yourself or to explain your version."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": system_message},
                  {"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"].strip()

def chat(agent1, agent2, agent3, n_turns=1):
    conversation_history = []
    for i in range(n_turns):
        agent1_message = send_message(f"{agent2} and {agent3}, {conversation_history}", temperature=1.8, role="Poet", max_tokens=80)
        conversation_history.append({"role": "user", "content": f"{agent1}: {agent1_message}"})
        wrapped_print(f"{agent1}: {agent1_message}")

        # Remove the Poet's message from the conversation history for Agent 3
        agent3_conversation_history = [msg for msg in conversation_history if msg["content"].startswith(agent1 + ":")]

        agent2_message = send_message(f"{agent1} and {agent3}, {agent3_conversation_history}", temperature=0.1, role="Proof Reader", max_tokens=100)
        conversation_history.append({"role": "user", "content": f"{agent2}: {agent2_message}"})
        wrapped_print(f"{agent2}: {agent2_message}")

        agent3_message = send_message(f"{agent1} and {agent2}, {conversation_history}", temperature=0.6, role="Editor", max_tokens=150)
        conversation_history.append({"role": "user", "content": f"{agent3}: {agent3_message}"})
        wrapped_print(f"{agent3}: {agent3_message}")


# Simulate a conversation between three agents
chat("Poet", "Proof Reader", "Editor")
